In [1]:
import shuo
import numpy as np
import networkx as nx
import geopandas as gpd
from shuo.basics import FileNames, PickleDataWriter as pdw, UnionFind, FilesReader

File_Prefixes = ['Washington_DC', 
                 'Seattle_city_WA',
                 'Chicago_city_IL',
                 'Madison_county_AL', 
                 'Mobile_city_AL', 
                 'Napa_city_CA', 
                 'Redding_city_CA',
                 'Yuma_county_AZ']
budget = 0
current_index = 0
folder = 'data_processed'
file_prefix = File_Prefixes[current_index]
print(file_prefix)

N = FileNames(folder, file_prefix, budget)
F = FilesReader(folder, file_prefix, budget)

Washington_DC


In [2]:
'''
Load basic data
    1. Original graph data: undirected road network graph
    2. TAZ (Traffic Analysis Zone) data: set of polygons partioning the whole area
    3. Travel demand data: based on TAZ, whose entry (i, j) stores the travel volume from TAZ i to TAZ j
    4. Flood zone data: set of polygons with flooding info
'''

# Load original graph data
OG = nx.read_graphml(N.graph())
print('Nodes:', len(OG.nodes), 'Edges:', len(OG.edges))

# Load TAZ data
zone_df = gpd.read_file(N.zone())

# Load travel demand data (based on TAZ)
demand_mat = F.travel_demand_matrix()

# Load flood zone data
flood_df = gpd.read_file(N.flood_zone())

Nodes: 10095 Edges: 16401


In [3]:
'''
Compute flooded road network graph
'''

# Build flood polygons list
flood_polygons = []
for index, row in flood_df.iterrows():
    # If use FEMA data
    if row['ZONE_SUBTY'] == '0.2 PCT ANNUAL CHANCE FLOOD HAZARD':
    # If use EnviroAtlas data
    #if row['Flooded'] == 1:
        flood_polygons.append(row['geometry'])
print('Flood polygons:', len(flood_polygons))

# Compute and save flooded graph
recompute = False
if recompute:
    FG = shuo.graph_with_flood_info(OG, flood_polygons)[0]
    nx.write_graphml(FG, N.graph(-1))
    UG, AG = shuo.unaffected_and_affected_graph(FG)
    nx.write_graphml(UG, N.graph(1))
    nx.write_graphml(AG, N.graph(2))
else:
    FG = nx.read_graphml(N.graph(-1))
    UG = nx.read_graphml(N.graph(1))
    AG = nx.read_graphml(N.graph(2))

Flood polygons: 202


In [4]:
'''
Compute Mapping
    1. Btw indices and nodes
    2. Btw indices and zones
    3. Btw zones and nodes
    4. Btw zone indices and node indices
'''

# Indices and nodes mapping
recompute = False
if recompute:
    dict_index_to_node, dict_node_to_index = shuo.indices_and_nodes_mapping(OG)
    pdw(N.mapping(0)).write_with_truncating(dict_index_to_node)
    pdw(N.mapping(0)).write(dict_node_to_index)
else:
    dict_index_to_node, dict_node_to_index = F.mapping(0)
    
# Indices and zones mapping
recompute = False
if recompute:
    dict_index_to_zone, dict_zone_to_index = shuo.indices_and_zones_mapping_from_GDF(zone_df, 'TAZ')
    pdw(N.mapping(1)).write_with_truncating(dict_index_to_zone)
    pdw(N.mapping(1)).write(dict_zone_to_index)
else:
    dict_index_to_zone, dict_zone_to_index = F.mapping(1)
    
# Compute and save zones and nodes mapping
recompute = False
if recompute:
    dict_zone_to_nodes, dict_node_to_zones = shuo.zones_and_nodes_mapping_from_GDF(OG, zone_df, 'TAZ')
    pdw(N.mapping(2)).write_with_truncating(dict_zone_to_nodes)
    pdw(N.mapping(2)).write(dict_node_to_zones)
else:
    dict_zone_to_nodes, dict_node_to_zones = F.mapping(2)
    
# Compute and save zone indices and node indices mapping
recompute = False
if recompute:
    dict_zone_ind_to_node_incs, dict_node_ind_to_zone_incs = shuo.zone_indices_and_node_indices_mapping_from_GDF(OG, zone_df)
    pdw(N.mapping(3)).write_with_truncating(dict_zone_ind_to_node_incs)
    pdw(N.mapping(3)).write(dict_node_ind_to_zone_incs)
else:
    dict_zone_ind_to_node_incs, dict_node_ind_to_zone_incs = F.mapping(3)

In [5]:
'''
Compute node and zone distances matrix along with trips counts matrix 
'''

# Compute and save node distance matrix
recompute = False
if recompute:
    node_dist_mat = shuo.nodes_distances(OG, dict_node_to_index)
    np.save(N.node_distance_matrix(), node_dist_mat)
else:
    node_dist_mat = F.node_distance_matrix()
    
# Compute and save zone distance and trips count matrix
recompute = False
if recompute:
    trips_count_mat = shuo.zones_distances_and_trips_count(node_dist_mat, dict_index_to_node, dict_node_to_zones, dict_zone_to_index)[1]
    np.save(N.zone_trips_count_matrix(), trips_count_mat)
else:
    trips_count_mat = F.zone_trips_count_matrix()

In [6]:
'''
Compute node and zone distances matrix along with trips counts matrix under flooding
'''

# Compute and save node distance matrix under flooding
recompute = False
if recompute:
    node_dist_mat = shuo.nodes_distances(UG, dict_node_to_index)
    np.save(N.node_distance_matrix(1), node_dist_mat)
else:
    node_dist_mat = F.node_distance_matrix(1)
    
# Compute and save zone distance and trips count matrix under flooding
recompute = False
if recompute:
    trips_count_mat = shuo.zones_distances_and_trips_count(node_dist_mat, dict_index_to_node, dict_node_to_zones, dict_zone_to_index)[1]
    np.save(N.zone_trips_count_matrix(1), trips_count_mat)
else:
    trips_count_mat = F.zone_trips_count_matrix(1)

In [7]:
'''
Compute compressed graphs and travel demand matrix
'''

# Compute and save compressed unaffected and affected graph along with travel demand matrix
recompute = False
if recompute:
    CUG, CAG, compressed_demand_mat = shuo.compressed_graph(UG, AG, (trips_count_mat[0] + trips_count_mat[1]), demand_mat, (dict_node_to_index, dict_zone_ind_to_node_incs))
    nx.write_graphml(CUG, N.compressed_graph(1))
    nx.write_graphml(CAG, N.compressed_graph(2))
    np.save(N.travel_demand_matrix(1), compressed_demand_mat)
else:
    CUG = F.compressed_graph(1)
    CAG = F.compressed_graph(2)
    compressed_demand_mat = F.travel_demand_matrix(1)

print('Nodes in compressed graph:', compressed_demand_mat.shape[0], 'Candidate edges:', len(CAG.edges))
print('Original travel demand:', np.sum(demand_mat), 'Compressed travel demand:', np.sum(compressed_demand_mat))

Nodes in compressed graph: 284 Candidate edges: 441
Original travel demand: 644337.6419999999 Compressed travel demand: 636061.5209999442


# Other Functionalities for Future Use

In [8]:
# Print some info of flooded graph
print('There\'re', len(UG.edges), 'unaffected edges and', len(AG.edges), 'affected edges\nRatio of affected edges to total edges:', round((len(AG.edges) * 100) / len(FG.edges), 2))

uf = UnionFind(UG.nodes, UG.edges)
children = uf.children()

uf.show_info()
children_counts = []
small_CC_num = 0
for key, value in children.items():
    num = len(value)
    if num > 5:
        children_counts.append(num)
    else:
        small_CC_num += 1
children_counts.sort(reverse = True)

print('There\'re', len(children_counts), 'connected components of size > 5:')
print(children_counts)

There're 15700 unaffected edges and 701 affected edges
Ratio of affected edges to total edges: 4.27
There're 284 connected components
There're 8 connected components of size > 5:
[7505, 2155, 34, 27, 16, 13, 12, 12]


In [9]:
# Construct a shapefile based on graphml
import fiona
import networkx as nx
import matplotlib as mpl
from matplotlib import pyplot as plt
from shapely.geometry import LineString, mapping
from shuo.basics import get_LineString

# Graphml to construct a shapefile
G = F.graph(-1)

# Output file path
file_path = folder + '/flooded.shp'

# Convert the nodes of road network into an easy-to-read format
points = {node: [float(data['x']), float(data['y'])] for node, data in G.nodes(data=True)}

# Create shapefile of colored edges
schema = {'geometry': 'LineString', 'properties':{'flood distance': 'float'}}
with fiona.open(file_path, 'w', 'ESRI Shapefile', schema, fiona.crs.from_epsg(4326)) as F:
    for u, v, d in G.edges(data=True):
        if d['flooded_distance'] == 0: continue
        elem = dict()
        line = get_LineString(d['geometry']) if 'geometry' in d else LineString([points[u], points[v]])
        elem['geometry'] = mapping(line)
        elem['properties'] = dict()
        elem['properties']['flood distance'] = d['flooded_distance']

        F.write(elem)